In [1]:
import numpy as np 
import pandas as pd 
import requests
from tqdm import tqdm 
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [191]:
driver = webdriver.Chrome(ChromeDriverManager().install())
link = 'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true'

driver.get(link)
soup = BeautifulSoup(driver.page_source,'html.parser')

/tmp/ipykernel_4085/1664324720.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [192]:
data = []

link = 'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true'

for pages in tqdm(range(2,401)):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    for info in soup.find_all('div',{'class' : 'a-section a-spacing-base'}):
        # name of the product 
        name = info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-title-instructions-style').find('h2').text
        # rating obtained by that product 
        try:
            if ' ' in info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text:
                stars = float(info.find('div',class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small').find('div',class_ = 'a-section a-spacing-none a-spacing-top-micro').find('span').find('i').text[:3])
            else:
                stars= 'not available'
        except:
            stars = 'not available'

        # price of the product 
        try:
            sell_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price').find('span').text[1:].replace(',',''))
        except:
            sell_price = 0
        try:
            mark_price = int(info.find('div',class_ = 'a-section a-spacing-none a-spacing-top-small s-price-instructions-style').find('a').find('span',class_ = 'a-price a-text-price').find('span',class_ = 'a-offscreen').text[1:].replace(',',''))
        except:
            mark_price = 0


        # delivery date 
        try:
            delivered_on = info.find('div',class_ =  'a-row a-size-base a-color-secondary s-align-children-center').find('span',class_ = 'a-color-base a-text-bold').text
        except:
            delivered_on = 'Data not available'

        data.append([name,stars,sell_price,mark_price,delivered_on])
        
    link = f'https://www.amazon.in/s?i=computers&rh=n%3A976392031&fs=true&page={pages}&qid=1679465284&ref=sr_pg_{pages}'

#         break

    # data

100%|█████████████████████████████████████████| 399/399 [37:22<00:00,  5.62s/it]


In [193]:
product = pd.DataFrame(data,columns =['name','stars','sell_price','mark_price','delivery_date'])
product.to_csv('product.csv',index = False)

In [194]:
df = pd.read_csv('product.csv')
df.head()

,name,stars,sell_price,mark_price,delivery_date
0,SHINING SUN VISION Smart LED Vision Chart 19 i...,3.5,19499,30000,"Wednesday, 29 March"
1,"Regatech Compatible for Gateway NV53A46U, NV53...",not available,799,2499,"Tuesday, 28 March"
2,"Regatech Compatible for Hp G62-A70SQ, G62-A70S...",not available,699,2499,"Tuesday, 28 March"
3,boAt Airdopes 141 Bluetooth Truly Wireless in ...,3.9,0,0,"Tomorrow, 23 March"
4,boAt BassHeads 100 in-Ear Wired Headphones wit...,4.1,379,999,"Today, 22 March"


In [195]:
df.shape

(13075, 5)